In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv('input/train.csv')
test_df = pd.read_csv('input/test.csv')
all_data_df = pd.concat([train_df,test_df])

/home/whale/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False




In [3]:
all_data_df['date'] = pd.to_datetime(all_data_df['date'])
all_data_df['month'] = all_data_df['date'].dt.month
all_data_df['weekday'] = all_data_df['date'].dt.dayofweek
all_data_df['week_of_year']  = all_data_df.date.dt.weekofyear

In [4]:
all_data_df['date'] = all_data_df['date'] - pd.to_datetime('2013-01-01')

In [5]:
all_data_df['date'] = all_data_df['date'].apply(lambda x: x.days)

In [6]:
all_data_df["median-store_item-month"] = all_data_df.groupby(["month","item","store"])["sales"].transform("median")
all_data_df["median-store_item-week"] = all_data_df.groupby(["week_of_year","item","store"])["sales"].transform("median")

all_data_df["mean-store_item-month"] = all_data_df.groupby(["week_of_year","item","store"])["sales"].transform("mean")
all_data_df["mean-store_item-week"] = all_data_df.groupby(["week_of_year","item","store"])["sales"].transform("mean")

all_data_df["item-month-sum"] = all_data_df.groupby(["month","item"])["sales"].transform("sum")

all_data_df["store-month-sum"] = all_data_df.groupby(["month","store"])["sales"].transform("sum")

In [10]:
all_data_df['store_item_shifted-90'] = all_data_df.groupby(["item","store"])['sales'].transform(lambda x:x.shift(90)) #3 months ago
all_data_df['store_item_shifted-122'] = all_data_df.groupby(["item","store"])['sales'].transform(lambda x:x.shift(122)) #4 months ago
all_data_df['store_item_shifted-180'] = all_data_df.groupby(["item","store"])['sales'].transform(lambda x:x.shift(180)) #6 months ago

In [11]:
all_data_df = all_data_df[all_data_df['date'] > 180]

In [17]:
col = all_data_df.columns.drop(['id', 'sales'])

In [18]:
y = 'sales'
train_df = all_data_df.loc[~all_data_df.sales.isna()]
test_df = all_data_df.loc[all_data_df.sales.isna()]

train_x, train_cv, train_y, train_y_cv = train_test_split(train_df[col],train_df[y], test_size=0.15, random_state=42)

In [19]:
random_forest = RandomForestRegressor(n_estimators=100)
random_forest.fit(train_x, train_y)
acc_random_forest = round(random_forest.score(train_cv, train_y_cv) * 100, 2)
acc_random_forest

93.32

In [20]:
random_forest.fit(train_df[col], train_df[y])
y_test = random_forest.predict(test_df[col])
sample = pd.read_csv('input/sample_submission.csv')
sample['sales'] = y_test
sample.to_csv('simple_starter.csv', index=False)